In [1]:
from edc import print_info
print_info("geodb-sharing")


***Notebook Title***  
GeoDB: Sharing Data

***Notebook Description***  
Euro Data Cube GeoDB: Sharing Data


***Notebook Dependencies***  
This notebook requires an active subscription to:
* EDC GeoDB
* EDC EOxHub Workspace


In [2]:
from edc import check_compatibility
check_compatibility("user-2022.07-00", dependencies=["GEODB"])



---------

The following environment variables are available:

* `GEODB_AUTH_AUD`, `GEODB_AUTH_CLIENT_ID`, `GEODB_AUTH_DOMAIN`, `GEODB_API_SERVER_URL`, `GEODB_AUTH_CLIENT_SECRET`, `GEODB_API_SERVER_PORT`


## Sharing Data

In [3]:
from xcube_geodb.core.geodb import GeoDBClient

In [4]:
geodb = GeoDBClient()
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

In [5]:
ds = geodb.get_my_collections()

if ds[(ds.database == geodb.whoami) & (ds.collection == 'land_use')].collection.count() == 0:
    # Have a look at fiona feature schema
    collections = {
            "land_use": 
            {
                "crs": 3794,
                "properties": 
                {
                    "RABA_PID": "float", 
                    "RABA_ID": "float", 
                    "D_OD": "date"
                }
            }
        }


    geodb.create_collections(collections)
    import geopandas
    import os
    gdf = geopandas.read_file(os.path.expanduser("~/.shared/notebooks/eurodatacube/notebooks/curated/data/sample/land_use.shp"))
    geodb.insert_into_collection('land_use', gdf)

### Publish a Collection to the World

In [6]:
geodb.list_my_grants()

,collection,grantee
0,2021-10-04-test-3,postgres
1,2021-10-04-test-3,postgres
2,2021-10-04-test-3,postgres
3,2021-10-04-test-3,postgres
4,2021-10-04-test-3,postgres
5,2021-10-04-test-3,postgres
6,2021-10-04-test-3,postgres
7,2021-10-04-test-1234,postgres
8,2021-10-04-test-1234,postgres
9,2021-10-04-test-1234,postgres


In [7]:
geodb.publish_collection("land_use")

### Accessing Collection as a different User

Please be aware that you cannot change the user easily within a Notebook. Here, we changed the credentials in the background. The following cells will not run and are for demonstration only.

In [8]:
geodb = GeoDBClient()
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

### Revoke access

In [9]:
geodb.list_my_grants()

,collection,grantee
0,2021-10-04-test-3,postgres
1,2021-10-04-test-3,postgres
2,2021-10-04-test-3,postgres
3,2021-10-04-test-3,postgres
4,2021-10-04-test-3,postgres
5,2021-10-04-test-3,postgres
6,2021-10-04-test-3,postgres
7,2021-10-04-test-1234,postgres
8,2021-10-04-test-1234,postgres
9,2021-10-04-test-1234,postgres


In [10]:
geodb.unpublish_collection("land_use")

In [11]:
geodb.list_my_grants()

,collection,grantee
0,2021-10-04-test-3,postgres
1,2021-10-04-test-3,postgres
2,2021-10-04-test-3,postgres
3,2021-10-04-test-3,postgres
4,2021-10-04-test-3,postgres
5,2021-10-04-test-3,postgres
6,2021-10-04-test-3,postgres
7,2021-10-04-test-1234,postgres
8,2021-10-04-test-1234,postgres
9,2021-10-04-test-1234,postgres


In [12]:
gdf = geodb.get_collection_by_bbox(collection="land_use", bbox=(452750.0, 88909.549, 464000.0, 102486.299),
                comparison_mode="contains", bbox_crs=3794, limit=2, offset=10)
gdf

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,11,2022-06-30T10:03:14.741129+00:00,None,"POLYGON ((460137.998 95628.898, 460111.001 956...",5983161,1100,2019-03-11
1,12,2022-06-30T10:03:14.741129+00:00,None,"POLYGON ((453673.609 91328.224, 453678.929 913...",5983074,1600,2019-03-26
